<h3>Executive Summary</h3>
Inter Galactic Medium (IGM) absorption
1. The effect occurs for wavelengths shorter than 1216 &#8491; at their source and at a red shift greater than (about) 6.
2. The apparent magnitudes of any given galaxy after the "IGM correction" will either be less or the same as the uncorrected values.  That is, the effect is one of absorption only.

This notebook analyses the results of a TAO lightcone from the Millennium/SAGE simulation with the computed SED at selected "bands" and with the IGM absorption applied.

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

Define functions to analyse the IGM results

In [7]:
def find_selected_galaxies(z_target, delta_z, df):
    """Return galaxies of the dataframe df that have the z_target redshift within a tolerance of delta_z"""
    low_z = z_target - delta_z
    if low_z <= 0.0:
        low_z = 0.0
    
    hi_z = z_target + delta_z
    lyalpha_lambda = 1216.0 
    
    obs_lambda_low, obs_lambda_hi = (1 + low_z)*lyalpha_lambda, (1 + hi_z)*lyalpha_lambda
    return (df['Redshift_Observed'] >= low_z) & (df['Redshift_Observed'] <= hi_z)

In [8]:
def positive(diff):
    return diff > 0

In [9]:
def find_boosted_galaxies_after_absorption(filters_of_interest):
    # Assert that the Galaxy_ID's of the two files (with and without) are lined up
    min_galid_diff = min(without_igm_abs['Galaxy_ID'] - with_igm_abs['Galaxy_ID']) 
    max_galid_diff = max(without_igm_abs['Galaxy_ID'] - with_igm_abs['Galaxy_ID'])
    assert(min_galid_diff == 0)
    assert(max_galid_diff == 0)
    ngal = len(without_igm_abs['Galaxy_ID'])
    nboosted = 0
    for filt in filters_of_interest:
        # with IGM absorption mags should be fainter -> more positive
        # therefore the diff should be negative if all went well.
        # so let's check for cases where the diff is positive
        diff = without_igm_abs[filt] - with_igm_abs[filt]
        xx = (np.where(diff > 0.0))[0]
        if len(xx) > 0:
            maxdiff = max(abs(np.asarray(filter(positive,diff))))
            nboosted = nboosted + len(xx)
            print("Out of {} galaxies, {} galaxies have boosted mags in filter {}".format(ngal,
                                                                                          len(xx),
                                                                                          filt))
            print("Max. absolute diff = {}".format(maxdiff))
            print("Printing some of the worst sample galaxies")
            print("###########################################")
            print("{:>12} {:>12} {:>12} {:>6}".format("GalaxyID",
                                           "Mag(IGM)",
                                           "Mag(no_IGM)",
                                            "Diff"))
            print("###########################################")
            nsample = 10
            for i in range(len(xx)):
                idx = xx[i]
                if (diff[idx] > maxdiff/2) :
                    print("{:18} {:12.4f} {:12.4f} {:6.4f} {} {}".format(
                        with_igm_abs['Galaxy_ID'][idx],
                        with_igm_abs[filt][idx],
                        without_igm_abs[filt][idx],diff[idx],
                        with_igm_abs["Redshift_Observed"][idx],
                        with_igm_abs["Redshift_Cosmological"][idx]))
                    nsample = nsample - 1
                    if nsample == 0:
                        break
      

    return nboosted

In [10]:
#with_igm_abs = pd.read_csv("data/sorted_rdb_withigm_much_more.csv")
#without_igm_abs = pd.read_csv("data/sorted_rdb_withoutigm_much_more.csv")
with_igm_abs = pd.read_csv("data/sorted_withigm_only.csv")
without_igm_abs = pd.read_csv("data/sorted_withoutigm_only.csv")
print("Reading the files with apparent and absolute mags")



Reading the files with apparent and absolute mags


In [11]:

igm_abs_mask = with_igm_abs["Redshift_Observed"] > 5.5
no_igm_abs_mask = without_igm_abs["Redshift_Observed"] > 5.5



In [12]:
len(with_igm_abs["Redshift_Observed"][igm_abs_mask])

12818

In [13]:
len(with_igm_abs["Redshift_Observed"])

162626

In [14]:
headers = with_igm_abs.dtypes.index

In [15]:
headers

Index([u'Total_Stellar_Mass', u'Bulge_Stellar_Mass', u'Black_Hole_Mass',
       u'Cold_Gas_Mass', u'Hot_Gas_Mass', u'Ejected_Gas_Mass',
       u'Intracluster_Stars_Mass', u'Metals_Total_Stellar_Mass',
       u'Metals_Bulge_Mass', u'Metals_Cold_Gas_Mass', u'Metals_Hot_Gas_Mass',
       u'Metals_Ejected_Gas_Mass', u'Metals_IntraCluster_Stars_Mass',
       u'Galaxy_Classification', u'Disk_Scale_Radius',
       u'Total_Star_Formation_Rate', u'Hot_Gas_Cooling_Rate',
       u'AGN_Heating_Rate', u'Quasar_BH_Accretion_Mass',
       u'Time_of_Last_Major_Merger', u'Time_of_Last_Minor_Merger',
       u'Supernova_Cold_Gas_Outflow_Rate', u'Mvir', u'Rvir', u'Vvir', u'Vmax',
       u'Velocity_Dispersion', u'X_Spin', u'Y_Spin', u'Z_Spin',
       u'Total_Particles', u'Central_Galaxy_Mvir', u'Subhalo_Mvir_at_Infall',
       u'Subhalo_Vvir_at_Infall', u'Subhalo_Vmax_at_Infall',
       u'Maximum_circular_velocity_of_the_halo', u'Flyby_Flag',
       u'Right_Ascension', u'Declination', u'Redshift_Cosmologic

In [16]:
z_target, delta_z = 2.0, 0.2
absorbed_galaxy_inds = find_selected_galaxies(z_target, delta_z, with_igm_abs)
unabsorbed_galaxy_inds = find_selected_galaxies(z_target, delta_z, without_igm_abs)

Check that the galaxy_id's match

In [17]:
assert(min(without_igm_abs[unabsorbed_galaxy_inds]['Galaxy_ID'] - with_igm_abs[absorbed_galaxy_inds]['Galaxy_ID'])==0)
assert(max(without_igm_abs[unabsorbed_galaxy_inds]['Galaxy_ID'] - with_igm_abs[absorbed_galaxy_inds]['Galaxy_ID'])==0)

In [18]:
z_target, delta_z = 9.0, 0.2
filters_of_interest_absolute = ['HST_WFC3_IR_F0.98M_Absolute', 'HST_WFC3_IR_F110W_Absolute',
                       'HST_WFC3_IR_F105W_Absolute', 'HST_WFC3_IR_F140W_Absolute',
                       'HST_WFC3_IR_F125W_Absolute', 'HST_WFC3_IR_F160W_Absolute',
                       'Johnson_B_Absolute', 'Johnson_V_Absolute', 'Johnson_R_Absolute',
                       'Johnson_I_Absolute', 'Johnson_J_Absolute', 'Johnson_H_Absolute',
                       'Johnson_K_Absolute', 'SDSS_u_Absolute', 'SDSS_g_Absolute',
                       'SDSS_r_Absolute', 'SDSS_i_Absolute', 'SDSS_z_Absolute',
                       'TwoMASS_J_Absolute', 'TwoMASS_H_Absolute', 'TwoMASS_Ks_Absolute']
filters_of_interest_apparent = ['HST_WFC3_IR_F0.98M_Apparent', 'HST_WFC3_IR_F110W_Apparent',
                       'HST_WFC3_IR_F105W_Apparent', 'HST_WFC3_IR_F140W_Apparent',
                       'HST_WFC3_IR_F125W_Apparent', 'HST_WFC3_IR_F160W_Apparent',
                       'Johnson_B_Apparent', 'Johnson_V_Apparent', 'Johnson_R_Apparent',
                       'Johnson_I_Apparent', 'Johnson_J_Apparent', 'Johnson_H_Apparent',
                       'Johnson_K_Apparent', 'SDSS_u_Apparent', 'SDSS_g_Apparent',
                       'SDSS_r_Apparent', 'SDSS_i_Apparent', 'SDSS_z_Apparent',
                       'TwoMASS_J_Apparent', 'TwoMASS_H_Apparent', 'TwoMASS_Ks_Apparent'
                      ]


In [19]:
assert(find_boosted_galaxies_after_absorption(filters_of_interest_absolute) == 0)
find_boosted_galaxies_after_absorption(filters_of_interest_apparent)

Out of 162626 galaxies, 1030 galaxies have boosted mags in filter HST_WFC3_IR_F0.98M_Apparent
Max. absolute diff = 0.0026
Printing some of the worst sample galaxies
###########################################
    GalaxyID     Mag(IGM)  Mag(no_IGM)   Diff
###########################################
251007967000000005      27.6157      27.6179 0.0022 6.27742 6.28249
251007932000000009      26.9712      26.9737 0.0025 6.27638 6.28105
251007882000000020      27.0876      27.0895 0.0019 6.24576 6.24746
251008102000000001      27.8750      27.8769 0.0019 6.25768 6.2563
251007867000000450      28.0488      28.0506 0.0018 6.25447 6.25687
251007905000000058      27.5095      27.5116 0.0021 6.28163 6.28301
251007947000000000      26.9690      26.9716 0.0026 6.27564 6.27967
251008002000000000      27.3349      27.3368 0.0019 6.24739 6.24901
251007919000000053      27.0447      27.0471 0.0024 6.27213 6.27351
251008464000000002      27.0579      27.0601 0.0022 6.26 6.2606
Out of 162626 galaxies, 55

4452